# OpenSpiel: Deep CFR in PyTorch
In this notebook we present an example of how to use OpenSpiel with PyTorch as the learning library. This is possible since most of the components in OpenSpiel are agnostic of the learning library used.

In this notebook we implement the Deep Counterfactual Regret algorithm in PyTorch on the game of Kuhn Poker. Adapted from [this example](https://github.com/deepmind/open_spiel/blob/master/open_spiel/python/examples/deep_cfr.py).


## Build and Install OpenSpiel
Clone the latest version of OpenSpiel from Github.

In [ ]:
#@title
%cd /usr/local
!git clone https://github.com/deepmind/open_spiel;
%cd open_spiel

Install the requirements and build the library.

In [ ]:
#@title
!./install.sh
!pip install -r requirements.txt;
!mkdir -p /usr/local/open_spiel/build
%cd /usr/local/open_spiel/build
!CXX=g++ cmake -DPython_TARGET_VERSION=3.6 -DCMAKE_CXX_COMPILER=${CXX} ../open_spiel;
!make -j$(nproc);

Set up the PYTHONPATH to enable the Python bindings for the current instance.

In [ ]:
#@title
%cd /content
import sys
sys.path.append('/usr/local/open_spiel')
sys.path.append('/usr/local/open_spiel/build/python')

In [ ]:
!pip install open_spiel

     |████████████████████████████████| 4.2 MB 6.1 MB/s 


## Import dependencies

In [ ]:
from open_spiel.python import policy
from open_spiel.python.algorithms import exploitability
from open_spiel.python.algorithms.deep_cfr import AdvantageMemory, StrategyMemory, ReservoirBuffer
import pyspiel
from absl import logging
import collections
import numpy as np

import torch
import torch.nn as nn

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
def reset_params(m):
  if isinstance(m, nn.Linear):
    m.reset_parameters()

# Defining fully-connected network to use as policy and advantage network
class MLP(nn.Module):
  def __init__(self, layers):
    """
      Creates a MLP with hidden units defined by `layers`.
    """
    super(MLP, self).__init__()
    layers = [nn.Linear(layers[i], layers[i+1]) for i in range(len(layers[:-1]))]
    self._model = nn.Sequential(*layers)
    
  def forward(self, input):
    return self._model(input)
  
  def reset(self):
    self._model.apply(reset_params)

## Implement DeepCFR
We follow the structure in the [Tensorflow implementation](https://github.com/deepmind/open_spiel/blob/master/open_spiel/python/algorithms/deep_cfr.py), rewiting the optimization steps in PyTorch. 

In [ ]:
class DeepCFR(policy.Policy):
  """Implements a solver for the Deep CFR Algorithm with PyTorch.

  See https://arxiv.org/abs/1811.00164.

  Define all networks and sampling buffers/memories.  Derive losses & learning
  steps. Initialize the game state and algorithmic variables.

  Note: batch sizes default to `None` implying that training over the full
        dataset in memory is done by default.  To sample from the memories you
        may set these values to something less than the full capacity of the
        memory.
  """

  def __init__(self,
               game,
               policy_network_layers=(256, 256),
               advantage_network_layers=(128, 128),
               num_iterations: int = 100,
               num_traversals: int = 20,
               learning_rate:float = 1e-4,
               batch_size_advantage=None,
               batch_size_strategy=None,
               memory_capacity: int =int(1e6),
               policy_network_train_steps: int = 1,
               advantage_network_train_steps: int = 1,
               reinitialize_advantage_networks: bool = True):
    """Initialize the Deep CFR algorithm.

    Args:
      game: Open Spiel game.
      policy_network_layers: (list[int]) Layer sizes of strategy net MLP.
      advantage_network_layers: (list[int]) Layer sizes of advantage net MLP.
      num_iterations: (int) Number of training iterations.
      num_traversals: (int) Number of traversals per iteration.
      learning_rate: (float) Learning rate.
      batch_size_advantage: (int or None) Batch size to sample from advantage
        memories.
      batch_size_strategy: (int or None) Batch size to sample from strategy
        memories.
      memory_capacity: Number af samples that can be stored in memory.
      policy_network_train_steps: Number of policy network training steps (per
        iteration).
      advantage_network_train_steps: Number of advantage network training steps
        (per iteration).
      reinitialize_advantage_networks: Whether to re-initialize the
        advantage network before training on each iteration.
    """
    all_players = list(range(game.num_players()))
    super(DeepCFR, self).__init__(game, all_players)
    self._game = game
    if game.get_type().dynamics == pyspiel.GameType.Dynamics.SIMULTANEOUS:
      # `_traverse_game_tree` does not take into account this option.
      raise ValueError("Simulatenous games are not supported.")
    self._batch_size_advantage = batch_size_advantage
    self._batch_size_strategy = batch_size_strategy
    self._policy_network_train_steps = policy_network_train_steps
    self._advantage_network_train_steps = advantage_network_train_steps
    self._num_players = game.num_players()
    self._root_node = self._game.new_initial_state()
    self._embedding_size = len(
        self._root_node.information_state_tensor(0))
    self._num_iterations = num_iterations
    self._num_traversals = num_traversals
    self._reinitialize_advantage_networks = reinitialize_advantage_networks
    self._num_actions = game.num_distinct_actions()
    self._iteration = 1

    # Define strategy network, loss & memory.
    self._strategy_memories = ReservoirBuffer(memory_capacity)
    self._policy_network = MLP(
      [self._embedding_size] + list(policy_network_layers) + [self._num_actions]) ## policy_network是不按照每个player分的
    # Illegal actions are handled in the traversal code where expected payoff
    # and sampled regret is computed from the advantage networks.
    self._policy_sm = nn.Softmax(dim=-1)
    self._loss_policy = nn.MSELoss()
    self._optimizer_policy = torch.optim.Adam(
                  self._policy_network.parameters(), lr=learning_rate)

    # Define advantage network, loss & memory. (One per player)
    self._advantage_memories = [
        ReservoirBuffer(memory_capacity) for _ in range(self._num_players)
    ]
    self._advantage_networks = [ ## advantage_network是按照每个player分的
        MLP([self._embedding_size] + list(advantage_network_layers) + 
            [self._num_actions])
        for _ in range(self._num_players)
    ]
    self._loss_advantages = nn.MSELoss(reduction='mean')
    self._optimizer_advantages = []
    for p in range(self._num_players):
      self._optimizer_advantages.append(torch.optim.Adam(
            self._advantage_networks[p].parameters(), lr=learning_rate))

  @property
  def advantage_buffers(self):
    return self._advantage_memories

  @property
  def strategy_buffer(self):
    return self._strategy_memories

  def clear_advantage_buffers(self):
    for p in range(self._num_players):
      self._advantage_memories[p].clear()

  def reinitialize_advantage_network(self, player):
    self._advantage_networks[player].reset()

  def reinitialize_advantage_networks(self):
    for p in range(self._num_players):
      self.reinitialize_advantage_network(p)

  def solve(self):
    """Solution logic for Deep CFR.

    Traverses the game tree, while storing the transitions for training
    advantage and policy networks. 

    Returns:
      1. (nn.Module) Instance of the trained policy network for inference. 
      2. (list of floats) Advantage network losses for 
        each player during each iteration.
      3. (float) Policy loss.   
    """
    advantage_losses = collections.defaultdict(list)
    for _ in range(self._num_iterations):
      for p in range(self._num_players):
        for _ in range(self._num_traversals):
          self._traverse_game_tree(self._root_node, p)
        if self._reinitialize_advantage_networks:
          # Re-initialize advantage network for player and train from scratch.
          self.reinitialize_advantage_network(p)
        # Re-initialize advantage networks and train from scratch.
        advantage_losses[p].append(self._learn_advantage_network(p))
      self._iteration += 1
      # Train policy network.
    policy_loss = self._learn_strategy_network()
    return self._policy_network, advantage_losses, policy_loss

  def _traverse_game_tree(self, state, player):
    """Performs a traversal of the game tree.

    Over a traversal the advantage and strategy memories are populated with
    computed advantage values and matched regrets respectively.

    Args:
      state: Current OpenSpiel game state.
      player: (int) Player index for this traversal.

    Returns:
      (float) Recursively returns expected payoffs for each action.
    """
    expected_payoff = collections.defaultdict(float)
    if state.is_terminal():
      # Terminal state get returns.
      return state.returns()[player]
    elif state.is_chance_node(): #chance_node 应该是 在该轮既不是当前player，也不是其他player做动作，而是环境的变化
      # If this is a chance node, sample an action
      action = np.random.choice([i[0] for i in state.chance_outcomes()])
      return self._traverse_game_tree(state.child(action), player)
    elif state.current_player() == player:
      sampled_regret = collections.defaultdict(float)
      # Update the policy over the info set & actions via regret matching.
      advantages, strategy = self._sample_action_from_advantage(state, player) ## 通过advantage网络，生成的遗憾，以及通过遗憾匹配来生成策略
      for action in state.legal_actions():
        expected_payoff[action] = self._traverse_game_tree( 
            state.child(action), player) #expected_payoff 是在执行某个action后，得到的payoff
      for action in state.legal_actions():
        sampled_regret[action] = expected_payoff[action]
        for a_ in state.legal_actions():
          sampled_regret[action] -= strategy[a_] * expected_payoff[a_] # 某个action的expected_payoff 减去总payoff的期望 等于该动作的遗憾值
      sampled_regret_arr = [0] * self._num_actions 
      for action in sampled_regret:
        sampled_regret_arr[action] = sampled_regret[action] # sampled_regret_arr 保持size为n_action,把非法动作加上去
      self._advantage_memories[player].add( 
          AdvantageMemory(state.information_state_tensor(),  
                          self._iteration, sampled_regret_arr, action)) #只是在自己回合内，会记录遗憾的真实值
      return max(expected_payoff.values()) #最终返回最大的expected_payoff
    else:
      other_player = state.current_player()
      _, strategy = self._sample_action_from_advantage(state, other_player) #通过advantage网络获得别人的策略
      # Recompute distribution for numerical errors.
      probs = np.array(strategy)
      probs /= probs.sum()
      sampled_action = np.random.choice(range(self._num_actions), p=probs) #策略选择，根据概率分布随机选择一个动作
      self._strategy_memories.add(
          StrategyMemory(
              state.information_state_tensor(other_player), #记录以下stragymeory
              self._iteration, strategy))
      return self._traverse_game_tree(state.child(sampled_action), player)

  def _sample_action_from_advantage(self, state, player):
    """Returns an info state policy by applying regret-matching.

    Args:
      state: Current OpenSpiel game state.
      player: (int) Player index over which to compute regrets.

    Returns:
      1. (list) Advantage values for info state actions indexed by action.
      2. (list) Matched regrets, prob for actions indexed by action.
    """
    info_state = state.information_state_tensor(player)
    legal_actions = state.legal_actions(player)
    with torch.no_grad():
      state_tensor = torch.FloatTensor(np.expand_dims(info_state, axis=0))
      advantages = self._advantage_networks[player](state_tensor)[0].numpy()
    advantages = [max(0., advantage) for advantage in advantages] #advantage网络是直接算的遗憾
    cumulative_regret = np.sum(
                            [advantages[action] for action in legal_actions])
    matched_regrets = np.array([0.] * self._num_actions) #遗憾匹配
    for action in legal_actions:
      if cumulative_regret > 0.:
        matched_regrets[action] = advantages[action] / cumulative_regret
      else:
        matched_regrets[action] = 1 / self._num_actions
    return advantages, matched_regrets

  def action_probabilities(self, state):
    """Computes the action probabilities for the current player 
      in the given state.
    
    Args:
      state: (pyspiel.State)
    
    Returns:
      (dict) action probabilities for a single batch."""
    cur_player = state.current_player()
    legal_actions = state.legal_actions(cur_player)
    info_state_vector = np.array(state.information_state_tensor())
    if len(info_state_vector.shape) == 1:
      info_state_vector = np.expand_dims(info_state_vector, axis=0)
    with torch.no_grad():
      logits = self._policy_network(torch.FloatTensor(info_state_vector))
      probs = self._policy_sm(logits).numpy()
    return {action: probs[0][action] for action in legal_actions}

  def _learn_advantage_network(self, player):
    """Compute the loss on sampled transitions and perform a Q-network update.

    If there are not enough elements in the buffer, no loss is computed and
    `None` is returned instead.

    Args:
      player: (int) player index.

    Returns:
      (float) The average loss over the advantage network.
    """
    for _ in range(self._advantage_network_train_steps):

      if self._batch_size_advantage:
        if self._batch_size_advantage > len(self._advantage_memories[player]):
          ## Skip if there aren't enough samples
          return None
        samples = self._advantage_memories[player].sample(
            self._batch_size_advantage)
      else:
        samples = self._advantage_memories[player]
      info_states = []
      advantages = []
      iterations = []
      for s in samples:
        info_states.append(s.info_state) #信息集
        advantages.append(s.advantage) #直接计算的遗憾
        iterations.append([s.iteration])
      # Ensure some samples have been gathered.
      if not info_states:
        return None
      self._optimizer_advantages[player].zero_grad()
      advantages = torch.FloatTensor(np.array(advantages))
      iters = torch.FloatTensor(np.sqrt(np.array(iterations)))
      outputs = self._advantage_networks[player](
                        torch.FloatTensor(np.array(info_states)))
      loss_advantages = self._loss_advantages(iters * outputs, 
                                              iters * advantages) #刚开始学习得快，随着迭代加深速度越来越慢
      loss_advantages.backward()
      self._optimizer_advantages[player].step()

    return loss_advantages.detach().numpy()

  def _learn_strategy_network(self):
    """Compute the loss over the strategy network.

    Returns:
      (float) The average loss obtained on this batch of transitions or `None`.
    """
    for _ in range(self._policy_network_train_steps):
      if self._batch_size_strategy:
        if self._batch_size_strategy > len(self._strategy_memories):
          ## Skip if there aren't enough samples
          return None
        samples = self._strategy_memories.sample(self._batch_size_strategy)
      else:
        samples = self._strategy_memories
      info_states = []
      action_probs = []
      iterations = []
      for s in samples:
        info_states.append(s.info_state)
        action_probs.append(s.strategy_action_probs)
        iterations.append([s.iteration])

      self._optimizer_policy.zero_grad()
      iters = torch.FloatTensor(np.sqrt(np.array(iterations)))
      ac_probs = torch.FloatTensor(np.array(np.squeeze(action_probs)))
      logits = self._policy_network(torch.FloatTensor(np.array(info_states)))
      outputs = self._policy_sm(logits)
      loss_strategy = self._loss_policy(iters * outputs, iters * ac_probs)
      loss_strategy.backward()
      self._optimizer_policy.step()

    return loss_strategy.detach().numpy()


## Train the model

In [ ]:
game = pyspiel.load_game('kuhn_poker')

In [ ]:
solver = DeepCFR(game,
      policy_network_layers=(32, 32),
      advantage_network_layers=(16, 16),
      num_iterations=400,
      num_traversals=40,
      learning_rate=1e-3,
      batch_size_advantage=None,
      batch_size_strategy=None,
      memory_capacity=1e7)

In [ ]:
_, advantage_losses, policy_loss = solver.solve()

In [ ]:
advantage_losses

defaultdict(list,
            {0: [array(1.7545124, dtype=float32),
              array(2.0155113, dtype=float32),
              array(2.6392589, dtype=float32),
              array(2.7119348, dtype=float32),
              array(3.733528, dtype=float32),
              array(4.1558113, dtype=float32),
              array(5.0745916, dtype=float32),
              array(5.6861076, dtype=float32),
              array(5.9701643, dtype=float32),
              array(5.751646, dtype=float32),
              array(6.4409947, dtype=float32),
              array(6.1743984, dtype=float32),
              array(7.021242, dtype=float32),
              array(6.871388, dtype=float32),
              array(6.898635, dtype=float32),
              array(8.097608, dtype=float32),
              array(8.232702, dtype=float32),
              array(9.259057, dtype=float32),
              array(8.570297, dtype=float32),
              array(9.564455, dtype=float32),
              array(8.287276, dtype=float32),
   

In [ ]:
for player, losses in list(advantage_losses.items()):
  print("Advantage for player:", player,
                losses[:2] + ["..."] + losses[-2:])
  print("Advantage Buffer Size for player", player,
                len(solver.advantage_buffers[player]))
print("Strategy Buffer Size:",
              len(solver.strategy_buffer))
print("Final policy loss:", policy_loss)
conv = exploitability.nash_conv(
    game,
    policy.tabular_policy_from_callable(game, solver.action_probabilities))
print("Deep CFR - NashConv:", conv)

Advantage for player: 0 [array(1.7545124, dtype=float32), array(2.0155113, dtype=float32), '...', array(206.09352, dtype=float32), array(206.96756, dtype=float32)]
Advantage Buffer Size for player 0 24348
Advantage for player: 1 [array(0.81504995, dtype=float32), array(1.7980549, dtype=float32), '...', array(242.28423, dtype=float32), array(270.8141, dtype=float32)]
Advantage Buffer Size for player 1 16000
Strategy Buffer Size: 56067
Final policy loss: 30.021711
Deep CFR - NashConv: 0.8816775822389887
